In [1]:
import numpy as np
from collections import deque
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, initializers, regularizers
from AtomasWrapper import AtomasWrapper

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
gpu_name = tf.test.gpu_device_name()
print(gpu_name)

Num GPUs Available:  1
/device:GPU:0


In [2]:
# simple
def create_q_model():

    state = layers.Input(shape = (21,))

    layer1 = layers.Dense(
        units = 128, 
        activation = "relu",
        kernel_initializer = "he_normal",
        bias_initializer = initializers.Constant(0.01),
        bias_regularizer = regularizers.L2(1e-4),
        activity_regularizer = regularizers.L2(1e-5)
    )(state)

    bn1 = layers.BatchNormalization()(layer1)

    layer2 = layers.Dense(
        units = 128, 
        activation = "relu",
        kernel_initializer = "he_normal",
        bias_initializer = initializers.Constant(0.01),
        bias_regularizer = regularizers.L2(1e-4),
        activity_regularizer = regularizers.L2(1e-5)
    )(bn1)

    bn2 = layers.BatchNormalization()(layer2)

    layer3 = layers.Dense(
        units = 128, 
        activation = "relu",
        kernel_initializer = "he_normal",
        bias_initializer = initializers.Constant(0.01),
        bias_regularizer = regularizers.L2(1e-4),
        activity_regularizer = regularizers.L2(1e-5)
    )(bn2)

    bn3 = layers.BatchNormalization()(layer3)

    q_value = layers.Dense(
        units = 18, 
        activation = "linear",
        kernel_initializer = "he_normal",
        bias_initializer = initializers.Constant(0.01),
        bias_regularizer = regularizers.L2(1e-4),
        activity_regularizer = regularizers.L2(1e-5)
    )(bn3)

    return keras.Model(inputs = state, outputs = q_value)



In [3]:
model = create_q_model()
print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 21)]              0         
_________________________________________________________________
dense (Dense)                (None, 128)               2816      
_________________________________________________________________
batch_normalization (BatchNo (None, 128)               512       
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512       
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
batch_normalization_2 (Batch (None, 128)               512   

In [4]:
# training procedure adapted from Yale's S&DS 365: Intermediate Machine Learning, Assignment 4.3.

NUM_ACTIONS = 18

GAMMA = 0.3           # decay rate of past observations
STEP_SIZE = 1e-4        # step size
OBSERVE = 10000         # timesteps to observe before training
TRAINING = 10000000     # timesteps of observing + training
REPLAY_MEMORY = 10000   # number of previous transitions to remember
BATCH_SIZE = 64         # size of each batch
EPSILON = 0.2           # exploration probability

ILLEGAL_ACTION_Q_VALUE = -1000000

losses = []

In [5]:
def dql_atomas(model, optimizer, loss_function):

    # initiate game
    game = AtomasWrapper()

    # store the previous state, action and transitions
    history_data = deque()

    # get the first observation by checking the game
    current_state, num_legal_actions, reward, terminal = game.check()
  
    # training
    t = 0

    while t < TRAINING:

        if terminal or num_legal_actions == 0:
            game = AtomasWrapper()
            current_state, num_legal_actions, reward, terminal = game.check()
            if num_legal_actions == 0:
                print("WARNING: Invalid Ring Replaced")

        if np.random.rand(1)[0] < EPSILON:
            # random action
            action = np.random.choice(num_legal_actions)
        else:
            # compute the Q function
            current_state_tensor = tf.convert_to_tensor(current_state)
            current_state_tensor = tf.expand_dims(current_state_tensor, 0)
            q_value = model(current_state_tensor, training = True)
          
            # greedy action   
            action = tf.argmax(q_value[0][:num_legal_actions]).numpy()

        # take the action and observe the reward and the next state
        
        action_vec = np.zeros(NUM_ACTIONS)
        action_vec[action] = 1
        next_state, num_legal_actions, reward, terminal = game.step(action_vec)

        # store the observation
        history_data.append((current_state, action, reward, next_state, 
                            terminal, num_legal_actions))
        if len(history_data) > REPLAY_MEMORY:
            history_data.popleft()  # discard old data


        # train if done observing
        if t > OBSERVE:

            # sample a batch
            batch = random.sample(history_data, BATCH_SIZE)
            state_sample = np.array([d[0] for d in batch])
            action_sample = np.array([d[1] for d in batch])
            reward_sample = np.array([d[2] for d in batch])
            state_next_sample = np.array([d[3] for d in batch])
            terminal_sample = np.array([d[4] for d in batch])
            legal_actions_sample = np.array([d[5] for d in batch])

            # compute the updated Q-values for the samples
            future_rewards = model(tf.convert_to_tensor(state_next_sample), training = True).numpy()

            # only consider q-values of legal actions
            for batch_idx, rewards in enumerate(future_rewards):
                for action_idx, reward_val in enumerate(rewards):
                    if action_idx >= legal_actions_sample[batch_idx]:
                        reward_val = ILLEGAL_ACTION_Q_VALUE
            
            terminal_tensor = tf.convert_to_tensor([float(s) for s in terminal_sample])   
            updated_q_value = reward_sample + (GAMMA * tf.reduce_max(future_rewards, axis = 1))

            # If final frame set the last value to -1
            updated_q_value = updated_q_value * (1 - terminal_tensor) - (terminal_tensor)

            # train the model on the states and updated Q-values
            with tf.GradientTape() as tape:

                # compute the current Q-values for the samples
                current_q_value = model(state_sample, training = True)
                mask = tf.one_hot(action_sample, NUM_ACTIONS)
                current_q_value = tf.reduce_sum(tf.multiply(current_q_value, mask), axis = 1)

                # compute the loss
                loss = loss_function(updated_q_value, current_q_value)

            # backpropagation
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

        else:
            loss = 0

        # update current state and counter
        current_state = next_state
        t += 1
        losses.append(loss)

        if t % 500 == 0: # originally 500
            print(f"STEP {t} | PHASE {'observe' if t <= OBSERVE else 'train'}", 
                  f"| ACTION {action} | REWARD {reward} | LOSS {loss}")
            if t > 10000 and t % 10000 == 0:
                model.save("dqn_model")
                wrapper = AtomasWrapper()
                wrapper.activate()


In [6]:
def train_atomas(start_from_ckpt = False, ckpt_path = None):

    np.random.seed(37)

    if start_from_ckpt:
        # if you want to start from a checkpoint
        model = keras.models.load_model("dqn_model")
    else:
        model = create_q_model()

    # specify the optimizer and loss function
    optimizer = keras.optimizers.SGD(learning_rate = STEP_SIZE, clipnorm = 1.0)
    loss_function = keras.losses.MeanSquaredError()

    # train model
    dql_atomas(model = model, optimizer = optimizer, loss_function = loss_function)

In [7]:
with tf.device(gpu_name):
    train_atomas(start_from_ckpt = True)

STEP 500 | PHASE observe | ACTION 0 | REWARD 0 | LOSS 0
STEP 1000 | PHASE observe | ACTION 0 | REWARD 0 | LOSS 0
STEP 1500 | PHASE observe | ACTION 0 | REWARD 0 | LOSS 0
STEP 2000 | PHASE observe | ACTION 0 | REWARD 0 | LOSS 0
STEP 2500 | PHASE observe | ACTION 0 | REWARD 0 | LOSS 0
STEP 3000 | PHASE observe | ACTION 0 | REWARD 0.0 | LOSS 0
STEP 3500 | PHASE observe | ACTION 6 | REWARD 0 | LOSS 0
STEP 4000 | PHASE observe | ACTION 0 | REWARD 0 | LOSS 0
STEP 4500 | PHASE observe | ACTION 0 | REWARD 0 | LOSS 0
STEP 5000 | PHASE observe | ACTION 0 | REWARD 0 | LOSS 0
STEP 5500 | PHASE observe | ACTION 6 | REWARD 0.0 | LOSS 0
STEP 6000 | PHASE observe | ACTION 0 | REWARD 0 | LOSS 0
STEP 6500 | PHASE observe | ACTION 0 | REWARD 0 | LOSS 0
STEP 7000 | PHASE observe | ACTION 0 | REWARD 0 | LOSS 0
STEP 7500 | PHASE observe | ACTION 0 | REWARD 0 | LOSS 0
STEP 8000 | PHASE observe | ACTION 8 | REWARD 0 | LOSS 0
STEP 8500 | PHASE observe | ACTION 0 | REWARD 3 | LOSS 0
STEP 9000 | PHASE observe | 

KeyboardInterrupt: 